# **Задание 1. Дискретная марковская цепь**

**Условие:** у нас три состояния $S = {0, 1, 2}$ и на каждом шаге из состояния $i$ мы с вероятностью $1 - \beta$ остаемся в том эе состоянии, а с вероятностью $\beta = 0.1$ переходим равновероятно в любое другое состояние

**1. Построение переходной матрицы $P$**

Для трех состояний матрица $P$ имеет размер $3 \times 3$. Элемент $P_{ij} = q(x_i = j | x_{t1} = 1)$.

- Если $i = j$, то:
$$P_{ij} = 1 - \beta = 0.9$$

- Если $i \ne j$, то из $i$ перейти в любую из двух других состояний с общей вероятностью $\beta$, то есть

$$P_{ij} = \frac{\beta}{2} = \frac{0.1}{2} = 0.05$$

Следовательно:

$$ P =
\left(
\begin{matrix}  
0.9 & 0.05 & 0.05\\  
0.05 & 0.9 & 0.05 \\
0.05 & 0.05 & 0.9 
\end{matrix}
\right)$$ 

**2. Нахожждение станционарного распределения $\pi$**

Станционарное распределение $\pi = (\pi_0, \pi_1, \pi_2)$ удовлетворяет

$$\pi = \pi P,   \sum_{i=0}^2\pi_i = 1.$$


То есть система уровнений:

$$
\begin{cases}  
    \pi_0 = 0.9\pi_0 + 0.05\pi_1 + 0.05\pi_2 \\  
    \pi_0 = 0.05\pi_0 + 0.9\pi_1 + 0.05\pi_2 \\  
    \pi_0 = 0.05\pi_0 + 0.05\pi_1 + 0.9\pi_2 \\  
    \pi_0 + \pi_1 + \pi_2 = 1  
\end{cases}  
$$

Из симметрии видно, что $\pi_0 = \pi_1 = \pi_2$. Тогда

$$3\pi_0 = 1  =>  \pi_0 = \frac{1}{3}$$

Итак,

$$\pi = \left(\frac{1}{3},\frac{1}{3},\frac{1}{3}\right)$$



---------------------------------
**Выводы:**
- Переходная матрица

$$ P =
\left(
\begin{matrix}  
0.9 & 0.05 & 0.05\\  
0.05 & 0.9 & 0.05 \\
0.05 & 0.05 & 0.9 
\end{matrix}
\right)$$ 

- Станционарное распределение

$$\pi = \left(\frac{1}{3},\frac{1}{3},\frac{1}{3}\right)$$

# **Задание 2: Прямая диффузия на MNIST**

**Цель:** 
Научиться **добавлять шум к изображениям** шаг за шагом, чтобы симулировать *forward process* в DDPM

**DDPM (Denoising Diffusion Probalistic Model)** - это тип генеративной модели предложенной в статье [Ho et al., 2020](https://arxiv.org/abs/2006.11239), которая учится **генерировать данные (например изображения) из случайного шума**, проходя через серию обученных шагов по "отшумлению"

In [1]:
import torch

In [ ]:
def q_sample(x0, t, betas):
    '''
    Добавление шума к изображению x0 на шаге t
        Args:
            - x0: [batch_size, C, H, W] - исходное изображение
            - t: [batch_size] или scalar - текущий шаг
            - betas: тензор [T] - шумовые коэффициенты
    '''
    alpha = 1.0 - betas
    alpha_cumprod = torch.cumprod(alpha, dim=0)     # [T]

    # Берём sqrt(alpha_cumprod[t]) и sqrt(1 - alpha_cumprod[t])
    sqrt_alpha_bar = alpha_cumprod[t] ** 0.5
    sqrt_one__alpha_bar = (1 - alpha_cumprod[t]) ** 0.5
